In [1]:
%load_ext iminizinc

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns

# import mplfinance as mpf
%matplotlib inline

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
plt.rcParams['lines.linewidth'] = 0.5 



<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.7.5, build 891740483
Copyright (C) 2014-2023 Monash University, NICTA, Data61


In [2]:
from minizinc import Instance, Model, Solver, Status as mzStatus
import nest_asyncio
nest_asyncio.apply()

In [5]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .1, .02
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')
    
def load_candles(pair, timeframe):
    odf = pd.read_json(f'../../freq-user-data/data/binance/{pair}-{timeframe}.json').dropna()
    odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

    odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
    # df.index = df['time']
    # df.set_index('time', drop=True, inplace=True)
    odf['idate'] = odf.date.dt.strftime('%Y%m%d')
    odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
    # df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
    # df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
    # df.set_index('time')
    odf = odf.sort_index()
    return odf



def add_indicators(wdf, hold, lag, ):
    wdf['rfd'] = wdf.low.rolling(hold).min().shift(-hold)
    wdf['rfu'] = wdf.high.rolling(hold).max().shift(-hold)
    wdf['lr_fu'] = np.log(wdf.rfu / wdf.close)
    wdf['lr_fd'] = np.log(wdf.rfd / wdf.close)
    
    wdf=wdf.join(wdf[['lr_fu', 'lr_fd']].apply(lambda x: x.multiply(100).round(2)).add_prefix('p'))
    # wdf=wdf.join(wdf[['plr_fu', 'plr_fd', 'plr_wiu', 'plr_wid']].apply(lambda x: pd.qcut(x,ncuts)).add_prefix('q'))

    # wdf['qmprfu'] = pd.qcut(wdf.mprfu, 10)
    return wdf

def call_mzn_model(model_name,model_params):
    with open(f'pars-{model_name}.dzn.json', 'w+') as f: f.write(json.dumps(model_params, indent=2))

    # print('Model params:', model_params)

    mzn_model = Model(f'{model_name}.mzn')
    gecode = Solver.lookup("gecode")
    instance = Instance(gecode, mzn_model)

    for k,v in model_params.items(): instance[k] = v
    result = instance.solve()

    return result


In [6]:
odf = load_candles('BTC_USDT', '8h')

In [1]:

### Ranges
nest_asyncio.apply()

strategy = 'stats-ranges'
strategy_params_json = f'./par-{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})

dlen = len(odf)


sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')

sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
sl_lag_n = IntSlider(value=5,min=1,max=50,step=1,description='lag_n',continuous_update=False)
# tx_max_rat = FloatText(value=0.05,min=0.01,max=0.1,step=0.001,description='max_rat',continuous_update=False)
sl_rat = FloatRangeSlider(value=(0.02,0.05),min=0.01,max=0.2,step=0.001,description='sl_rat',continuous_update=False)
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir},
    "lag_n": {"wdg": sl_lag_n},
    # "max_rat": {"wdg": tx_max_rat},
    "rat": {"wdg": sl_rat}
}
all_params = {
    'w2log': {'wdg': sl_n2}, 'w': {'wdg': sl_w},
    **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w]),
    widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in all_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf,glag_n, min_rat, max_rat = 0,0,0,0
mzn_result = None
sdf, spa_mins, spa_maxs, i_spa_begs, i_spa_sigs, ix_spa_begs, ix_spa_sigs = 0,0,0,0,0,0,0
i_hi_ends, i_lo_ends, ix_hi_ends, ix_lo_ends = 0,0,0,0
def printer(
        w2log, w, hold, candles, mdir, lag_n, rat
):
    
    global wdf,glag_n, min_rat, max_rat
    global mzn_result
    global sdf, spa_mins, spa_maxs, i_spa_begs, i_spa_sigs, ix_spa_begs, ix_spa_sigs
    global i_hi_ends, i_lo_ends, ix_hi_ends, ix_lo_ends

    glag_n = lag_n
    min_rat, max_rat = rat
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = odf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    print(f'N={len(wdf)}; Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')
    # print(f'lag_n={lag_n}; min_rat={min_rat}; max_rat={max_rat}\n')

    sdf = pd.DataFrame({'close': wdf.close.astype(int)}
    ).assign(ba_max=lambda x: x['close'].rolling(lag_n).max().bfill().astype(int), ba_min = lambda x: x['close'].rolling(lag_n).min().bfill().astype(int)
    ).assign(ba_rat = lambda df:df['ba_max']/df['ba_min'] - 1
    ).assign(spa_in=lambda x:((x['ba_rat'] >= min_rat) & (x['ba_rat'] <= max_rat))
    ).assign(spa_sig=lambda x: x['spa_in'] & (~x['spa_in']).shift()
    ).assign(spa_beg=lambda x:x['spa_sig'].shift(-(lag_n-1))
    )
    sdf.to_csv('test.csv')

    i_spa_begs, i_spa_sigs = [sdf[c].fillna(False).values.nonzero()[0] for c in ['spa_beg','spa_sig']]
    ix_spa_begs, ix_spa_sigs = [wdf.index[c] for c in [i_spa_begs, i_spa_sigs]]
    spa_mins = [int(sdf.loc[ix_spa_begs[i]:ix_spa_sigs[i]].close.min()) for i in range(len(ix_spa_sigs))]
    spa_maxs = [int(sdf.loc[ix_spa_begs[i]:ix_spa_sigs[i]].close.max()) for i in range(len(ix_spa_sigs))]
    # if len(spa_mins) > 0: spa_mins[-1] = int(sdf.close.max())
    # if len(spa_maxs) > 0: spa_maxs[-1] = int(sdf.close.min())
    # print(f'i_spa_begs={i_spa_begs}, i_spa_sigs={i_spa_sigs}\n')
    # print(f'spa_mins={spa_mins}, spa_maxs={spa_maxs}\n')
    rats = np.array([])
    if len(ix_spa_sigs) > 0:
        rats = np.array(spa_maxs) / spa_mins - 1
        # , rats={rats}
        print(f'Spans={len(ix_spa_sigs)}, ratsrange={rats.min(),rats.max()}\n')
    else:
        print('No spans')

    ###/ MiniZinc to find span endings
    # min_close, max_close
    rsdf = sdf.loc[::-1]    
    fu_mins, fu_maxs = rsdf.close.cummin()[::-1], rsdf.close.cummax()[::-1]
    fu_mins[-1] = int(sdf.close.min())
    fu_maxs[-1] = int(sdf.close.max())
    mzn_result = call_mzn_model('span-endings',{
        'i_spa_sigs': (i_spa_sigs+1).tolist(),
        'closes' : sdf.close.values.tolist(),
        'ba_mins' : sdf.ba_min.bfill().values.tolist(),
        'ba_maxs' : sdf.ba_max.bfill().values.tolist(),
        'spa_mins': spa_mins, 
        'spa_maxs': spa_maxs,
        'cmin': int(sdf.close.min()), 'cmax': int(sdf.close.max()),
        'min_to_end': fu_mins.values.tolist(), 'max_to_end': fu_maxs.values.tolist()
    })
    print(f'mzn_result: {mzn_result}')
    ix_hi_ends, ix_lo_ends = [], []
    if mzn_result.status == mzStatus.SATISFIED:
        i_lo_ends = (np.array(mzn_result.solution.i_lo_ends)-1)
        i_hi_ends = (np.array(mzn_result.solution.i_hi_ends)-1)
        if len(i_hi_ends) > 0: 
            ix_lo_ends = wdf.index[i_lo_ends] 
            ix_hi_ends = wdf.index[i_hi_ends] 
    # print(f'i_spa_ends={i_spa_ends}')
    ###\ Minizinc


    plt.close('all')
    fig = plt.figure(figsize=(9,3))

    (ax1,ax2, ax3) = fig.subplots(3, 1, height_ratios=[3,1,1], sharex=True);

    plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
    # ax1.plot(fu_mins,c='g')
    # ax1.plot(fu_maxs,c='r')
    # ax1.plot(sdf.ba_min, c='g', lw=0.2); ax1.plot(sdf.ba_max, c='r', lw=0.2)
    # sdf.ba_max.plot(ax=ax1, c='r')
    if len(ix_spa_sigs) > 0:
        spa_lquargs = {'lw':0.3,'color':'m'}
        ax1.vlines(ix_spa_begs, spa_mins, spa_maxs,**{**spa_lquargs})
        # ax1.vlines(ix_spa_sigs, spa_mins, spa_maxs,**{**spa_lquargs})
        # ax1.vlines(ix_spa_ends, spa_mins, spa_maxs,**{**spa_lquargs, 'linestyle':'--'})
        ax1.vlines(ix_spa_sigs, spa_mins, spa_maxs,**{**spa_lquargs, 'lw':1, 'alpha':0.7})
        ax1.hlines(spa_mins, ix_spa_begs, ix_spa_sigs, **spa_lquargs)
        ax1.hlines(spa_maxs, ix_spa_begs, ix_spa_sigs, **spa_lquargs)
        ax1.hlines(spa_mins, ix_spa_sigs, ix_lo_ends, **{**spa_lquargs, 'linestyle':'--'})
        ax1.hlines(spa_maxs, ix_spa_sigs, ix_hi_ends, **{**spa_lquargs, 'linestyle':'--'})

    plt.show()
    
out = widgets.interactive_output(printer, {
        **{k : v['wdg'] for k,v in all_params.items()}
    });
x = display(ui, out);


NameError: name 'nest_asyncio' is not defined

In [544]:
mzn_result

Result(status=<Status.SATISFIED: 5>, solution=Solution(i_spa_ends=[15, 24, 34, 45, 62, 108, 115], _checker=''), statistics={'paths': 0, 'flatBoolVars': 190, 'flatIntVars': 4, 'flatSetVars': 7, 'flatBoolConstraints': 95, 'flatIntConstraints': 95, 'flatSetConstraints': 500, 'evaluatedReifiedConstraints': 588, 'method': 'satisfy', 'flatTime': datetime.timedelta(microseconds=40106), 'time': datetime.timedelta(microseconds=48000), 'initTime': datetime.timedelta(microseconds=1002), 'solveTime': datetime.timedelta(microseconds=79), 'solutions': 1, 'variables': 204, 'propagators': 26, 'propagations': 1038, 'nodes': 5, 'failures': 0, 'restarts': 0, 'peakDepth': 4, 'nSolutions': 1})

In [574]:
spa_mins, ix_spa_sigs, ix_hi_ends

([8316, 8316, 9197, 9485, 9771],
 DatetimeIndex(['2020-01-25 00:00:00', '2020-01-26 16:00:00',
                '2020-02-05 08:00:00', '2020-02-07 08:00:00',
                '2020-02-09 00:00:00'],
               dtype='datetime64[ns]', name='date', freq=None),
 [])

In [604]:
[sdf.loc[ix_spa_begs[i]:ix_spa_sigs[i]].close.min() for i in range(len(ix_spa_sigs))]

[6153,
 6242,
 6711,
 7143,
 6766,
 6828,
 6699,
 6892,
 7027,
 6826,
 7482,
 7528,
 8620,
 8842,
 9018,
 9592,
 8650,
 9475,
 9316,
 9540,
 9068,
 9055,
 8720,
 9412,
 9468,
 9500,
 9719,
 9280,
 9342,
 9298,
 9399,
 9299,
 9259,
 9419,
 9173,
 9012,
 9135,
 9028,
 9257,
 9181,
 9123,
 9160,
 9302,
 9537,
 9593,
 10881,
 11071,
 11628,
 11392,
 11823,
 11726,
 11616,
 11586,
 11465,
 11618,
 10140,
 10113]